In [ ]:
pip install pandas

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 12.6 MB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 20.6 MB 19.5 MB/s eta 0:00:01
     |████████████████████████████████| 345 kB 26.3 MB/s eta 0:00:01
     |████████████████████████████████| 505 kB 44.2 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
!pip install openai --upgrade


  Attempting uninstall: openai
    Found existing installation: openai 1.13.3
    Uninstalling openai-1.13.3:
      Successfully uninstalled openai-1.13.3


In [ ]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 62 kB 3.7 MB/s eta 0:00:011
     |████████████████████████████████| 122 kB 17.2 MB/s eta 0:00:01
     |████████████████████████████████| 121 kB 32.9 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from openai import OpenAI
import pandas as pd
import os
import requests
import time
import concurrent.futures
import threading
import glob
# Load your dataset
df = pd.read_csv('merged_bfro.csv')

client = OpenAI(
    api_key="key here"
)

# rate_limit = 8.57

# Semaphore to control the rate of API requests
request_semaphore = threading.Semaphore(value=2)
last_request_time = [time.time()]

def generate_image(caption, image_path):
    # Acquire the semaphore to make an API request
    with request_semaphore:
        current_time = time.time()
        time_since_last_request = current_time - last_request_time[0]

        # Wait if the time since the last request is less than the rate limit
        # if time_since_last_request < rate_limit:
        #     time.sleep(rate_limit - time_since_last_request)

        # Make the API request
        response = client.images.generate(
            model="dall-e-2",
            prompt=caption,
            size="1024x1024",
            quality="standard",
            n=1,
        )
        last_request_time[0] = time.time()  # Update the time of the last request

    image_url = response.data[0].url

    # Download and save the image
    with open(image_path, 'wb') as f:
        f.write(requests.get(image_url).content)

def generate_image_with_fallback(caption, observed, witness_count, image_path):
    try:
        generate_image(caption, image_path)
    except Exception as e:
        print(f"Error generating image with full caption: {e}")
        try:
            fallback_caption = observed
            generate_image(fallback_caption, image_path)
        except Exception as e:
            print(f"Error generating image with 'Observed' column: {e}")
            final_caption = f"A bigfoot image with witnesses {witness_count}"
            generate_image(final_caption, image_path)

def process_row(row):
    if row['Report Type'] == 'Report':
        image_filename_base = f"{row['Id']}"
        image_path_pattern = os.path.join('images', f"{image_filename_base}.*")

        # Check if an image with the same Id already exists (either as PNG or JPEG)
        existing_images = glob.glob(image_path_pattern)
        if existing_images:
            print(f"Image for row {row['Id']} already exists. Skipping.")
            return existing_images[0]  # Return the path of the existing image

        # If no image exists, proceed with image generation
        image_filename = f"{image_filename_base}.jpeg"
        image_path = os.path.join('images', image_filename)

        observed_text = observed_text = str(row['Observed'])[:500]
  # Take only the first 3000 characters
        caption = f"If this is information about a bigfoot sighting, create an image based on this information: {observed_text} Witnesses: {row['Witness Count']}"
        generate_image_with_fallback(caption, observed_text, str(row['Witness Count']), image_path)
        print(f"Image for row {row['Id']} finished generating.")
        return image_path
    return None



# Create the 'images' folder if it doesn't exist
os.makedirs('images', exist_ok=True)

# Use ThreadPoolExecutor to process rows in parallel
with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
    future_to_row = {executor.submit(process_row, row): row for _, row in df.iterrows()}
    for future in concurrent.futures.as_completed(future_to_row):
        row = future_to_row[future]
        try:
            image_path = future.result()
            if image_path:
                df.at[row.name, 'Generated_Image_Path'] = image_path
        except Exception as exc:
            print(f"Row {row['Id']} generated an exception: {exc}")

# Throttling is handled within the generate_image function. Adjust according to your needs.

# Save the updated dataframe with the paths to the generated images
df.to_csv('task5_dataset.csv', index=False)

Image for row 8792 already exists. Skipping.
Image for row 13038 already exists. Skipping.
Image for row 1255 already exists. Skipping.
Image for row 11616 already exists. Skipping.
Image for row 637 already exists. Skipping.
Image for row 26604 already exists. Skipping.
Image for row 1257 already exists. Skipping.
Image for row 1995 already exists. Skipping.
Image for row 1258 already exists. Skipping.
Image for row 1256 already exists. Skipping.
Image for row 1259 already exists. Skipping.
Image for row 597 already exists. Skipping.
Image for row 1260 already exists. Skipping.
Image for row 7963 already exists. Skipping.
Image for row 8797 already exists. Skipping.
Image for row 2917 already exists. Skipping.
Image for row 25442 already exists. Skipping.
Image for row 23666 already exists. Skipping.
Image for row 9317 already exists. Skipping.
Image for row 1261 already exists. Skipping.
Image for row 1783 already exists. Skipping.
Image for row 26513 already exists. Skipping.
Image 